In [253]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from termcolor import colored
import requests
from bs4 import BeautifulSoup
from collections import Counter
pc = ['#FFB6C1', '#FFA07A', '#FFD700', '#98FB98', '#DDA0DD', '#87CEEB', '#F0E68C', '#FFE4E1', '#98FB98', '#FF69B4']


In [254]:
df = pd.read_csv('virat_odi_bat.csv')

In [255]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Unnamed: 9,Opposition,Ground,Start Date,Runs.1
0,101*,195,121,10,0,83.47,3,not out,1,NaN,v South Africa,Eden Gardens,5 Nov 2023,ODI # 4694
1,88,139,94,11,0,93.61,3,caught,1,NaN,v Sri Lanka,Wankhede,2 Nov 2023,ODI # 4690
2,0,12,9,0,0,0.00,3,caught,1,NaN,v England,Lucknow,29 Oct 2023,ODI # 4686
3,95,154,104,8,2,91.34,3,caught,2,NaN,v New Zealand,Dharamsala,22 Oct 2023,ODI # 4678
4,103*,113,97,6,4,106.18,3,not out,2,NaN,v Bangladesh,Pune,19 Oct 2023,ODI # 4674


In [256]:
df.pop(df.columns[13])

0      ODI # 4694
1      ODI # 4690
2      ODI # 4686
3      ODI # 4678
4      ODI # 4674
          ...    
284    ODI # 2756
285    ODI # 2755
286    ODI # 2750
287    ODI # 2745
288    ODI # 2742
Name: Runs.1, Length: 289, dtype: object

In [257]:
df.pop(df.columns[9])

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
284   NaN
285   NaN
286   NaN
287   NaN
288   NaN
Name: Unnamed: 9, Length: 289, dtype: float64

In [258]:
df.pop(df.columns[7])

0      not out
1       caught
2       caught
3       caught
4      not out
        ...   
284        lbw
285     bowled
286    run out
287     caught
288        lbw
Name: Dismissal, Length: 289, dtype: object

In [259]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date
0,101*,195,121,10,0,83.47,3,1,v South Africa,Eden Gardens,5 Nov 2023
1,88,139,94,11,0,93.61,3,1,v Sri Lanka,Wankhede,2 Nov 2023
2,0,12,9,0,0,0.00,3,1,v England,Lucknow,29 Oct 2023
3,95,154,104,8,2,91.34,3,2,v New Zealand,Dharamsala,22 Oct 2023
4,103*,113,97,6,4,106.18,3,2,v Bangladesh,Pune,19 Oct 2023


In [260]:
df.shape

(289, 11)

In [261]:
df = df[df['Runs'] != 'DNB']
df = df[df['Runs'] != 'TDNB']

In [262]:
df.shape

(277, 11)

In [263]:
indian_stadiums = ['Eden Gardens', 'Wankhede', 'Lucknow', 'Dharamsala','Pune','Ahmedabad','Delhi','Chennai','Rajkot'\
'Visakhapatnam','Indore','Hyderabad','Thiruvananthapuram','Raipur','Guwahati','Bengaluru','Cuttack','Nagpur'\
                   ,'Ranchi','Mohali']
df['Ground'] = df['Ground'].apply(lambda x: 1 if x in indian_stadiums else 0)

In [264]:
df['Start Date'] = pd.to_datetime(df['Start Date'], format='%d %b %Y').dt.year
df['Opposition'] = df['Opposition'].str[2:]

In [265]:
df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
df['BF'] = pd.to_numeric(df['BF'], errors='coerce')
df['4s'] = pd.to_numeric(df['4s'], errors='coerce')
df['6s'] = pd.to_numeric(df['6s'], errors='coerce')
df['SR'] = pd.to_numeric(df['SR'], errors='coerce')
df['Pos'] = pd.to_numeric(df['Pos'], errors='coerce')
df['Inns'] = pd.to_numeric(df['Inns'], errors='coerce')
df['NO'] = df['Runs'].str.endswith('*').astype(int)
df['Runs'] = df['Runs'].str.replace('*', '')
df['Runs'] = pd.to_numeric(df['Runs'], errors='coerce')

<ipython-input-265-c49f0468a10c>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Runs'] = df['Runs'].str.replace('*', '')


In [266]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,NO
0,101,195.0,121,10,0,83.47,3,1,South Africa,1,2023,1
1,88,139.0,94,11,0,93.61,3,1,Sri Lanka,1,2023,0
2,0,12.0,9,0,0,0.00,3,1,England,1,2023,0
3,95,154.0,104,8,2,91.34,3,2,New Zealand,1,2023,0
4,103,113.0,97,6,4,106.18,3,2,Bangladesh,1,2023,1


In [267]:
df.describe()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Ground,Start Date,NO
count,277.000000,241.000000,277.000000,277.000000,277.000000,276.000000,277.000000,277.000000,277.00000,277.000000,277.000000
mean,49.191336,72.887967,52.581227,4.606498,0.534296,80.726341,3.198556,1.548736,0.33213,2015.151625,0.158845
std,43.175949,57.846347,39.341842,4.539723,1.078318,36.432824,0.702169,0.498520,0.47183,4.178808,0.366193
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000,2008.000000,0.000000
25%,10.000000,20.000000,17.000000,1.000000,0.000000,63.625000,3.000000,1.000000,0.00000,2011.000000,0.000000
50%,37.000000,61.000000,47.000000,3.000000,0.000000,82.130000,3.000000,2.000000,0.00000,2015.000000,0.000000
75%,82.000000,120.000000,83.000000,7.000000,1.000000,100.000000,3.000000,2.000000,1.00000,2018.000000,0.000000
max,183.000000,220.000000,159.000000,22.000000,8.000000,209.090000,7.000000,2.000000,1.00000,2023.000000,1.000000


In [268]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(df['Opposition']))
df['Opposition'] = df['Opposition'].apply(lambda x: le.transform([x])[0])

In [269]:
df.describe()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,NO
count,277.000000,241.000000,277.000000,277.000000,277.000000,276.000000,277.000000,277.000000,277.000000,277.00000,277.000000,277.000000
mean,49.191336,72.887967,52.581227,4.606498,0.534296,80.726341,3.198556,1.548736,6.191336,0.33213,2015.151625,0.158845
std,43.175949,57.846347,39.341842,4.539723,1.078318,36.432824,0.702169,0.498520,3.636001,0.47183,4.178808,0.366193
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.00000,2008.000000,0.000000
25%,10.000000,20.000000,17.000000,1.000000,0.000000,63.625000,3.000000,1.000000,3.000000,0.00000,2011.000000,0.000000
50%,37.000000,61.000000,47.000000,3.000000,0.000000,82.130000,3.000000,2.000000,7.000000,0.00000,2015.000000,0.000000
75%,82.000000,120.000000,83.000000,7.000000,1.000000,100.000000,3.000000,2.000000,9.000000,1.00000,2018.000000,0.000000
max,183.000000,220.000000,159.000000,22.000000,8.000000,209.090000,7.000000,2.000000,12.000000,1.00000,2023.000000,1.000000


In [270]:
import numpy as np
conditions = [
    (df['Runs'] >= 100),
    (df['Runs'] >= 50)
]
values = [2, 1]
df['milestone'] = np.select(conditions, values, default=0)

In [271]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,NO,milestone
0,101,195.0,121,10,0,83.47,3,1,8,1,2023,1,2
1,88,139.0,94,11,0,93.61,3,1,9,1,2023,0,1
2,0,12.0,9,0,0,0.00,3,1,3,1,2023,0,0
3,95,154.0,104,8,2,91.34,3,2,6,1,2023,0,1
4,103,113.0,97,6,4,106.18,3,2,2,1,2023,1,2


Algorithm for batsman :
0.30*run_scored+0.05*notout_innigs+
0.20*bat_avg + 0.15*bat_sr+0.15*milestone_reaching_ability+ 0.10*no_of_4’s_6’s+0.05*high_score-0.05*no_of_zeroes
β_pos_score =max (α_batsmen_score_at_each_position)
4: γ_inningwise_score = 0.40*α_batsmen_score_first_inning+
0.60*α_batsmen_score_seocnd_inning
5: x_venue = 0.35*α_batsmen_score_home_matches+0.65*
α_batsmen_score_away_matches
6: y_opponent = 0.70*α_batsmen_score_strong_opponent+0.30*
α_batsmen_score_weak_opponent
7: w_yearwise = 0.20*α_batsmen_score_current_year+0.80*
α_batsmen_score_last_five_year
8:batting_score = 0.25*α_batsmen_score+0.10*β_pos_score+0.15*γ_inningwise_score+0.15*x_venue +0.15*
y_opponent+0.20*w_yearwise


In [272]:
def coun(data):
  ans = (data['Runs'].sum()*0.3) + (0.05*data[data['NO'] == 1].shape[0]) +(0.20*data['Runs'].mean())\
  +(data['SR'].mean()*0.15) + (15*(data[data['milestone'] == 1].shape[0]+data[data['milestone'] == 2].shape[0])/data.shape[0]) +\
  (0.10 * (data['4s'].sum()+data['6s'].sum())) + (0.05 * data['Runs'].max()) - (0.05 * data[data['Runs'] == 0].shape[0])
  return ans

In [273]:
def count(df):
  alpha_batsmen_score = coun(df)
  unp = df['Pos'].unique()
  beta_batsmen_score = 0
  for i in range(len(unp)):
    ans = coun(df[df['Pos'] == unp[i]])
    beta_batsmen_score = max (beta_batsmen_score,ans)
  gemma_batsmen_score = (0.40*coun(df[df['Inns'] == 1])) + (0.60*coun(df[df['Inns'] == 2]))
  x_venue = (0.35*coun(df[df['Ground'] == 1])) + (0.65*coun(df[df['Ground'] == 0]))
  y_opponent = (0.70*coun(df[df['Opposition'] == 3])) + (0.30*coun(df[df['Opposition'] == 7]))
  w_yearwise =(0.80*coun(df[(df['Start Date'] >= 2018) & (df['Start Date'] <= 2022)])) \
  + (0.20*coun(df[df['Start Date'] == 2023]))
  batting_score = (0.25*alpha_batsmen_score) + (0.10*beta_batsmen_score) +\
  (0.15*gemma_batsmen_score) + (0.15*x_venue) + (0.15*y_opponent) + (0.20*w_yearwise)
  return batting_score

In [274]:
ans = count(df)
print(ans)

2365.1347422228905


In [275]:
df2 = pd.read_csv('virat_odi_bow.csv')

In [276]:
df2.head()

,Overs,Mdns,Runs,Wkts,Econ,Pos,Inns,Unnamed: 7,Opposition,Ground,Start Date,Unnamed: 11
0,DNB,-,-,-,-,-,2,NaN,v Sri Lanka,Dambulla,18 Aug 2008,ODI # 2742
1,DNB,-,-,-,-,-,1,NaN,v Sri Lanka,Dambulla,20 Aug 2008,ODI # 2745
2,DNB,-,-,-,-,-,2,NaN,v Sri Lanka,Colombo (RPS),24 Aug 2008,ODI # 2750
3,DNB,-,-,-,-,-,2,NaN,v Sri Lanka,Colombo (RPS),27 Aug 2008,ODI # 2755
4,DNB,-,-,-,-,-,1,NaN,v Sri Lanka,Colombo (RPS),29 Aug 2008,ODI # 2756


In [277]:
df2.pop(df2.columns[11])

0      ODI # 2742
1      ODI # 2745
2      ODI # 2750
3      ODI # 2755
4      ODI # 2756
          ...    
284    ODI # 4674
285    ODI # 4678
286    ODI # 4686
287    ODI # 4690
288    ODI # 4694
Name: Unnamed: 11, Length: 289, dtype: object

In [278]:
df2.pop(df2.columns[7])

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
284   NaN
285   NaN
286   NaN
287   NaN
288   NaN
Name: Unnamed: 7, Length: 289, dtype: float64

In [279]:
df2 = df2[df2['Overs'] != 'DNB']
df2 = df2[df2['Overs'] != 'TDNB']

In [280]:
df2['Overs'] = pd.to_numeric(df2['Overs'], errors='coerce')
df2['Mdns'] = pd.to_numeric(df2['Mdns'], errors='coerce')
df2['Runs'] = pd.to_numeric(df2['Runs'], errors='coerce')
df2['Wkts'] = pd.to_numeric(df2['Wkts'], errors='coerce')
df2['Econ'] = pd.to_numeric(df2['Econ'], errors='coerce')
df2['Pos'] = pd.to_numeric(df2['Pos'], errors='coerce')
df2['Inns'] = pd.to_numeric(df2['Inns'], errors='coerce')
df2['Start Date'] = pd.to_datetime(df2['Start Date'], format='%d %b %Y').dt.year

In [281]:
df2.describe()

,Overs,Mdns,Runs,Wkts,Econ,Pos,Inns,Start Date
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,2.163265,0.020408,13.612245,0.081633,6.710204,5.551020,1.346939,2012.938776
std,1.388148,0.142857,7.947265,0.276642,2.942797,1.444259,0.480929,2.366504
min,0.200000,0.000000,1.000000,0.000000,3.000000,3.000000,1.000000,2009.000000
25%,1.000000,0.000000,7.000000,0.000000,5.140000,4.000000,1.000000,2011.000000
50%,2.000000,0.000000,13.000000,0.000000,6.000000,6.000000,1.000000,2013.000000
75%,3.000000,0.000000,17.000000,0.000000,7.330000,7.000000,2.000000,2014.000000
max,7.000000,1.000000,44.000000,1.000000,18.000000,8.000000,2.000000,2023.000000


In [282]:
df2.isnull().sum()

Overs         0
Mdns          0
Runs          0
Wkts          0
Econ          0
Pos           0
Inns          0
Opposition    0
Ground        0
Start Date    0
dtype: int64

In [283]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(df2['Opposition']))
df2['Opposition'] = df2['Opposition'].apply(lambda x: le.transform([x])[0])

In [284]:
indian_stadiums = ['Eden Gardens', 'Wankhede', 'Lucknow', 'Dharamsala','Pune','Ahmedabad','Delhi','Chennai','Rajkot'\
'Visakhapatnam','Indore','Hyderabad','Thiruvananthapuram','Raipur','Guwahati','Bengaluru','Cuttack','Nagpur'\
                   ,'Ranchi','Mohali']
df2['Ground'] = df2['Ground'].apply(lambda x: 1 if x in indian_stadiums else 0)

In [285]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 6 to 284
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Overs       49 non-null     float64
 1   Mdns        49 non-null     int64  
 2   Runs        49 non-null     int64  
 3   Wkts        49 non-null     int64  
 4   Econ        49 non-null     float64
 5   Pos         49 non-null     int64  
 6   Inns        49 non-null     int64  
 7   Opposition  49 non-null     int64  
 8   Ground      49 non-null     int64  
 9   Start Date  49 non-null     int64  
dtypes: float64(2), int64(8)
memory usage: 4.2 KB


In [286]:
def bowscore(data):
  cw = 1
  if data['Wkts'].sum() != 0 :
    cw = 1/data['Wkts'].sum()
  else :
    cw = 0
  ans = (data['Wkts'].sum()*0.3) + (0.20*data['Runs'].sum()*cw) + \
  (0.10*((((data['Overs'] * 10) % 10).astype(int))+ (data['Overs'].astype(int) * 6)).sum()) + \
  (0.15*data['Runs'].sum()/data['Overs'].sum()) + (0.10*(data[data['Wkts']==4].shape[0]+data[data['Wkts']==5].shape[0])) +\
  (0.05*data['Wkts'].max() + (0.10*data['Mdns'].sum()) )
  return ans

In [287]:
def count2(df):
  alpha_bow_score = bowscore(df)
  beta_bow_score = (0.40*bowscore(df[df['Inns'] == 1])) + (0.60*bowscore(df[df['Inns'] == 2]))
  x_venue = (0.40*bowscore(df[df['Ground'] == 1])) + (0.60*bowscore(df[df['Ground'] == 0]))
  y_opponent = (0.80*bowscore(df[df['Opposition'] == 3])) + (0.20*bowscore(df[df['Opposition'] == 7]))
  w_yearwise =(0.80*bowscore(df[(df['Start Date'] >= 2018) & (df['Start Date'] <= 2022)]))
  + (0.20*bowscore(df[df['Start Date'] == 2023]))
  bow_score = (0.30*alpha_bow_score) + (0.20*beta_bow_score) +\
  + (0.15*0) + (0.20*0) + (0.15*0)
  return bow_score

In [288]:
count2(df2)

<ipython-input-286-bfdb6115aef5>:9: RuntimeWarning: invalid value encountered in double_scalars
  (0.15*data['Runs'].sum()/data['Overs'].sum()) + (0.10*(data[data['Wkts']==4].shape[0]+data[data['Wkts']==5].shape[0])) +\


44.796570886389695

In [289]:
#Batting allrounder
batall = (count2(df2)*0.30) + (0.50*count(df)) + (0.20*(count(df)-count2(df2)))
print(batall)

1660.0739766446623


<ipython-input-286-bfdb6115aef5>:9: RuntimeWarning: invalid value encountered in double_scalars
  (0.15*data['Runs'].sum()/data['Overs'].sum()) + (0.10*(data[data['Wkts']==4].shape[0]+data[data['Wkts']==5].shape[0])) +\


In [290]:
#Bowl allrounder
bowall = (count2(df2)*0.45) + (0.35*count(df)) + (0.20*(count(df)-count2(df2)))
print(bowall)

<ipython-input-286-bfdb6115aef5>:9: RuntimeWarning: invalid value encountered in double_scalars
  (0.15*data['Runs'].sum()/data['Overs'].sum()) + (0.10*(data[data['Wkts']==4].shape[0]+data[data['Wkts']==5].shape[0])) +\


1312.023250944187


In [296]:
df3 = pd.read_csv('virat_odi_f.csv')

In [297]:
df3.head()

,Dis,Ct,St,Ct Wk,Ct Fi,Inns,Unnamed: 6,Opposition,Ground,Start Date,Unnamed: 10
0,0,0,0,0,0,2,NaN,v Sri Lanka,Dambulla,18 Aug 2008,ODI # 2742
1,3,3,0,0,3,1,NaN,v Sri Lanka,Dambulla,20 Aug 2008,ODI # 2745
2,0,0,0,0,0,2,NaN,v Sri Lanka,Colombo (RPS),24 Aug 2008,ODI # 2750
3,0,0,0,0,0,2,NaN,v Sri Lanka,Colombo (RPS),27 Aug 2008,ODI # 2755
4,0,0,0,0,0,1,NaN,v Sri Lanka,Colombo (RPS),29 Aug 2008,ODI # 2756


In [298]:
df3.pop(df3.columns[10])

0      ODI # 2742
1      ODI # 2745
2      ODI # 2750
3      ODI # 2755
4      ODI # 2756
          ...    
284    ODI # 4674
285    ODI # 4678
286    ODI # 4686
287    ODI # 4690
288    ODI # 4694
Name: Unnamed: 10, Length: 289, dtype: object

In [299]:
df3.pop(df3.columns[6])

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
284   NaN
285   NaN
286   NaN
287   NaN
288   NaN
Name: Unnamed: 6, Length: 289, dtype: float64

In [300]:
df3.pop(df3.columns[1])

0      0
1      3
2      0
3      0
4      0
      ..
284    0
285    2
286    0
287    0
288    0
Name: Ct, Length: 289, dtype: object

In [301]:
df3.pop(df3.columns[0])

0      0
1      3
2      0
3      0
4      0
      ..
284    0
285    2
286    0
287    0
288    0
Name: Dis, Length: 289, dtype: object

In [312]:
df3.describe()

,St,Ct Wk,Ct Fi,Opposition,Ground,Start Date
count,286.0,286.0,286.000000,289.000000,289.000000,289.000000
mean,0.0,0.0,0.524476,6.868512,0.325260,2015.152249
std,0.0,0.0,0.709160,4.110033,0.469284,4.225132
min,0.0,0.0,0.000000,0.000000,0.000000,2008.000000
25%,0.0,0.0,0.000000,3.000000,0.000000,2011.000000
50%,0.0,0.0,0.000000,8.000000,0.000000,2014.000000
75%,0.0,0.0,1.000000,10.000000,1.000000,2019.000000
max,0.0,0.0,3.000000,13.000000,1.000000,2023.000000


In [304]:
indian_stadiums = ['Eden Gardens', 'Wankhede', 'Lucknow', 'Dharamsala','Pune','Ahmedabad','Delhi','Chennai','Rajkot'\
'Visakhapatnam','Indore','Hyderabad','Thiruvananthapuram','Raipur','Guwahati','Bengaluru','Cuttack','Nagpur'\
                   ,'Ranchi','Mohali']
df3['Ground'] = df3['Ground'].apply(lambda x: 1 if x in indian_stadiums else 0)

In [310]:
df3.head()

,St,Ct Wk,Ct Fi,Inns,Opposition,Ground,Start Date
0,0,0,0,2,10,0,2008
1,0,0,3,1,10,0,2008
2,0,0,0,2,10,0,2008
3,0,0,0,2,10,0,2008
4,0,0,0,1,10,0,2008


In [305]:
df3['Start Date'] = pd.to_datetime(df3['Start Date'], format='%d %b %Y').dt.year

In [311]:
df3['St'] = pd.to_numeric(df3['St'], errors='coerce')
df3['Ct Wk'] = pd.to_numeric(df3['Ct Wk'], errors='coerce')
df3['Ct Fi'] = pd.to_numeric(df3['Ct Fi'], errors='coerce')

In [306]:
df3.isnull().sum()

St            0
Ct Wk         0
Ct Fi         0
Inns          0
Opposition    0
Ground        0
Start Date    0
dtype: int64

In [308]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(df3['Opposition']))
df3['Opposition'] = df3['Opposition'].apply(lambda x: le.transform([x])[0])

In [322]:
def wek(data):
  beta_wicket_keeping_ability = 0.70*data['Ct Wk'].sum() + 0.30*data['St'].sum()
  wicket_keeper_score = 0.45 * count(df) + 0.55*beta_wicket_keeping_ability
  return wicket_keeper_score

In [323]:
print(wek(df3))

1064.3106340003008
